In [3]:
import pandas as pd
data_path = "../data/open_source_8454_combine_short_description.csv"  # 替換為你的檔案路徑
df = pd.read_csv(data_path)
df

,Inbound Message,Label
0,login issue verified user details employee man...,0
1,outlook hello team meetings skype meetings etc...,0
2,cant log vpn cannot log vpn best,0
3,unable access tool page unable access tool page,0
4,skype error skype error,0
...,...,...
8449,emails coming mail good afternoon receiving em...,22
8450,telephony software issue telephony software issue,0
8451,vip windows password reset tifpdchb pedxruyf v...,0
8452,machine est funcionando unable access machine ...,44


# 十折驗證法

In [4]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}") 
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
print(f"Current GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9} GB")
print(f"Current GPU Memory Cached: {torch.cuda.memory_reserved() / 1e9} GB")

Using device: cuda
CUDA Available: True
CUDA Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Current GPU Memory Allocated: 0.0 GB
Current GPU Memory Cached: 0.0 GB


In [5]:
from transformers import BertTokenizer
from sklearn.model_selection import KFold
# 設定參數
MAX_LENGTH = 128
BATCH_SIZE = 16
EPOCHS = 20

# 初始化 BERT 的 tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 初始化數據
messages = df['Inbound Message'].tolist()
labels = df['Label'].tolist()

# KFold 初始化
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [6]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, messages, labels, tokenizer, max_length):
        self.messages = messages
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, idx):
        message = str(self.messages[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            message,
            max_length=self.max_length,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [7]:
# 定義訓練與評估函數
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    return total_loss / len(dataloader)

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np
import torch

def eval_model(model, dataloader, criterion, num_labels):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            total_loss += loss.item()

            probs = torch.softmax(logits, dim=1)
            _, preds = torch.max(logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    val_loss = total_loss / len(dataloader)
    val_accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)

    # **檢查驗證集是否包含兩個以上的類別**
    unique_labels = np.unique(all_labels)
    if len(unique_labels) > 1:
        all_labels_binarized = label_binarize(all_labels, classes=np.arange(num_labels))
        try:
            macro_auc = roc_auc_score(all_labels_binarized, all_probs, multi_class='ovo', average='macro')
            weighted_auc = roc_auc_score(all_labels_binarized, all_probs, multi_class='ovo', average='weighted')
        except ValueError as e:
            print(f"ROC AUC 計算失敗: {e}")
            macro_auc = None
            weighted_auc = None
    else:
        print(f"跳過 ROC AUC 計算，因為驗證集中只有一個類別: {unique_labels}")
        macro_auc = None
        weighted_auc = None

    return {
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "macro_f1": macro_f1,
        "balanced_accuracy": balanced_acc,
        "macro_auc": macro_auc,
        "weighted_auc": weighted_auc
    }


In [9]:
from sklearn.model_selection import KFold
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np

kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 進行 10 折交叉驗證
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(messages)):
    print(f"Fold {fold + 1}")

    train_messages = [messages[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_messages = [messages[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]

    train_dataset = CustomDataset(
        messages=train_messages,
        labels=train_labels,
        tokenizer=tokenizer,
        max_length=MAX_LENGTH
    )
    val_dataset = CustomDataset(
        messages=val_messages,
        labels=val_labels,
        tokenizer=tokenizer,
        max_length=MAX_LENGTH
    )
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    NUM_LABELS = len(set(train_labels) | set(val_labels))  # 修正 NUM_LABELS 計算
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=NUM_LABELS)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
    criterion = CrossEntropyLoss()

    patience = 3
    best_val_loss = float('inf')
    best_metrics = None
    epochs_without_improvement = 0
    
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        train_loss = train_epoch(model, train_dataloader, optimizer, criterion)
        metrics = eval_model(model, val_dataloader, criterion, num_labels=NUM_LABELS)
        print(metrics)
        
        if metrics['val_loss'] < best_val_loss:
            best_val_loss = metrics['val_loss']
            best_metrics = metrics
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
        
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break
    
    fold_results.append(best_metrics)

# 確保 fold_results 不為空
if len(fold_results) > 0:
    avg_metrics = {metric: np.mean([result[metric] for result in fold_results if result[metric] is not None]) for metric in fold_results[0].keys()}
    print("10-fold Cross-Validation Results:")
    for metric, value in avg_metrics.items():
        print(f"{metric}: {value}")
else:
    print("交叉驗證未成功執行，fold_results 為空")


Fold 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20


100%|██████████| 476/476 [03:19<00:00,  2.38it/s]


ROC AUC 計算失敗: Only one class present in y_true. ROC AUC score is not defined in that case.
{'val_loss': 1.6580788335710201, 'val_accuracy': 0.5910165484633569, 'precision': 0.42455818531370304, 'recall': 0.5910165484633569, 'f1': 0.4739585724757325, 'macro_f1': 0.09182643954932168, 'balanced_accuracy': 0.11180373872197664, 'macro_auc': None, 'weighted_auc': None}
Epoch 2/20


...

100%|██████████| 476/476 [03:21<00:00,  2.36it/s]


ROC AUC 計算失敗: Only one class present in y_true. ROC AUC score is not defined in that case.
{'val_loss': 1.3944833447348397, 'val_accuracy': 0.6615384615384615, 'precision': 0.6387406538539622, 'recall': 0.6615384615384615, 'f1': 0.6307347252642909, 'macro_f1': 0.31318845734551903, 'balanced_accuracy': 0.34304192284493235, 'macro_auc': None, 'weighted_auc': None}
Epoch 6/20


100%|██████████| 476/476 [03:21<00:00,  2.36it/s]


ROC AUC 計算失敗: Only one class present in y_true. ROC AUC score is not defined in that case.
{'val_loss': 1.405953770538546, 'val_accuracy': 0.6733727810650888, 'precision': 0.6627184330390584, 'recall': 0.6733727810650888, 'f1': 0.6532857885737835, 'macro_f1': 0.3610852726215593, 'balanced_accuracy': 0.3817911576662932, 'macro_auc': None, 'weighted_auc': None}
Early stopping triggered at epoch 6
10-fold Cross-Validation Results:
val_loss: 1.2628595225934713
val_accuracy: 0.6752990054135718
precision: 0.6460233981454737
recall: 0.6752990054135718
f1: 0.6433309241196437
macro_f1: 0.29793289268755824
balanced_accuracy: 0.30423134637688404
macro_auc: nan
weighted_auc: nan


D:\Anaconda3\envs\cuda\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\Anaconda3\envs\cuda\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [10]:
fold_results

[{'val_loss': 1.3080203724357318,
  'val_accuracy': 0.6666666666666666,
  'precision': 0.6337799187285923,
  'recall': 0.6666666666666666,
  'f1': 0.6277707038412302,
  'macro_f1': 0.3061860447780919,
  'balanced_accuracy': 0.30729855985174087,
  'macro_auc': None,
  'weighted_auc': None},
 {'val_loss': 1.2018907390675455,
  'val_accuracy': 0.6773049645390071,
  'precision': 0.6317207702262758,
  'recall': 0.6773049645390071,
  'f1': 0.6379441533170974,
  'macro_f1': 0.2589144372064198,
  'balanced_accuracy': 0.25124397117746217,
  'macro_auc': None,
  'weighted_auc': None},
 {'val_loss': 1.2572681313415743,
  'val_accuracy': 0.6903073286052009,
  'precision': 0.6696775898165531,
  'recall': 0.6903073286052009,
  'f1': 0.6685759735277139,
  'macro_f1': 0.297353605304868,
  'balanced_accuracy': 0.30713783067346867,
  'macro_auc': None,
  'weighted_auc': None},
 {'val_loss': 1.2255917149332334,
  'val_accuracy': 0.6891252955082743,
  'precision': 0.6687569224288511,
  'recall': 0.6891252